In [2]:
import pandas as pd
import numpy as np
import os
import sys
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
import torch.nn as nn
np.random.seed(42)
import torch
torch.manual_seed(42)
from model.model_mlp import train,predict
from scipy.stats import spearmanr

/home/laiminzhi/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.DataFrame({'x':np.arange(0,10),'y':np.arange(0,10)})
df.mean(axis=0)

x    4.5
y    4.5
dtype: float64

In [8]:
area = pd.read_csv('batch_results_rank_area.csv',index_col=0)
area = area.sort_values(by='Area',ascending=False)

In [6]:
path_train = '/home/laiminzhi/new_data/train_data/xy_data_2836/lstm_30_y1_label__new_uniform_no_ud_limit_h2/'
path_test = '/home/laiminzhi/new_data/test_data/xy_data_2836/lstm_30_y1_label__new_uniform_no_ud_limit_h2/'

train_x = np.load(path_train+'train_x.npy',allow_pickle=True)
valid_x = np.load(path_train+'valid_x.npy',allow_pickle=True)
train_y = np.load(path_train+'train_y.npy',allow_pickle=True)
test_x = np.load(path_test+'test_x.npy',allow_pickle=True)
test_y = np.load(path_test+'test_y.npy',allow_pickle=True)

In [2]:
xydata_path = '/home/laiminzhi/data/xydata/xy_data_2836.h5'
xy_2836 = pd.read_hdf(xydata_path)

In [ ]:
class no_udlimit_wmse(nn.Module):
    def __init__(self,alpha=0.01):
        super(no_udlimit_wmse, self).__init__()
        self.alpha = alpha

    def forward(self, predict, target, weight):
        """
        计算带权重的均方误差（WMSE）
        
        :param predict: 预测值，形状为 (code, 1)
        :param target: 真实值，形状为 (code, 1, 1)
        :param weight: 权重条件，形状为 (code, 1)
        :return: 加权均方误差
        """
        # 确保predict和target的形状一致
        target = target.squeeze()
        
        # 设置权重：如果weight的值为1，则权重设置为0.01，否则设置为1
        weights = torch.where(weight == 1, torch.tensor(self.alpha), torch.tensor(1.0))
        
        # 计算误差（差的平方）
        errors = (predict - target) ** 2
        
        # 应用权重
        weighted_errors = errors * weights
        
        # 计算WMSE
        wmse = weighted_errors.mean()
        
        return wmse

In [19]:
#读入涨停股票的数据
ud_limit_raw = pd.read_hdf(f"/home/laiminzhi/data/xydata/xy_data_2836_udlimit.h5")
ud_limit = pd.pivot_table(ud_limit_raw,index = "date",columns = "code",values = "ud_limit_h2")
#转换为numpy数组
ud_limit = np.array(ud_limit.values)

#weight为ud_limit当天的数据
weight = torch.tensor(ud_limit[29])

In [14]:
test_y_df = pd.read_hdf(path_test + 'test_y.h5')

In [21]:
timestep = 30
dates = test_y_df.index.get_level_values(0).unique().values[timestep-1:]
stocks = test_y_df.index.get_level_values(1).unique().values
test_y = np.load(path_test+'test_y.npy',allow_pickle=True)
test_y = test_y.transpose(1,2,0)
test_y = test_y[timestep-1:,:,:]
# 展平数组，移除最后一个维度
arr_flattened = test_y.squeeze(-1)

# 检查维度，确保对应的日期和股票数量与数组匹配
assert len(dates) == arr_flattened.shape[0], "日期数量不匹配"
assert len(stocks) == arr_flattened.shape[1], "股票数量不匹配"

# 填充 'y_hat' 列
for i, date in enumerate(dates):
    for j, stock in enumerate(stocks):
        # 只有当(date, stock)在DataFrame的索引中时，才进行赋值
        if (date, stock) in test_y_df.index:
            test_y_df.loc[(date, stock), 'y_hat'] = arr_flattened[i, j]

In [34]:
group = test_y_df.groupby(level=1)
for i,df in group:
    print(i)    
    #统计多少nan
    print(df['y_hat'].isna().sum())
    if i==40:
        break

000001.SZ
29
000002.SZ
29
000006.SZ
29
000009.SZ
29
000010.SZ
29
000011.SZ
29
000012.SZ
29
000014.SZ
29
000016.SZ
29
000017.SZ
29
000019.SZ
29
000021.SZ
29
000023.SZ
29
000025.SZ
29
000026.SZ
29
000027.SZ
29
000028.SZ
29
000030.SZ
29
000031.SZ
29
000032.SZ
29
000034.SZ
29
000035.SZ
29
000036.SZ
29
000037.SZ
29
000039.SZ
29
000040.SZ
29
000045.SZ
29
000049.SZ
29
000050.SZ
29
000055.SZ
29
000056.SZ
29
000058.SZ
29
000059.SZ
29
000060.SZ
29
000061.SZ
29
000062.SZ
29
000063.SZ
29
000065.SZ
29
000066.SZ
29
000069.SZ
29
000070.SZ
29
000078.SZ
29
000088.SZ
29
000089.SZ
29
000090.SZ
29
000099.SZ
29
000100.SZ
29
000150.SZ
29
000151.SZ
29
000153.SZ
29
000155.SZ
29
000156.SZ
29
000157.SZ
29
000158.SZ
29
000159.SZ
29
000166.SZ
29
000301.SZ
29
000333.SZ
29
000338.SZ
29
000400.SZ
29
000401.SZ
29
000402.SZ
29
000403.SZ
29
000404.SZ
29
000407.SZ
29
000408.SZ
29
000411.SZ
29
000416.SZ
29
000417.SZ
29
000423.SZ
29
000425.SZ
29
000426.SZ
29
000429.SZ
29
000488.SZ
29
000498.SZ
29
000501.SZ
29
000503.SZ
29

In [7]:
selected_feature = pd.read_csv('selected_feature.csv',index_col=0)['AlphaName'].to_list()

In [7]:
#导入数据
path = "/home/laiminzhi/wenbin/DL_stock_combo/data/xy_data/xy_data_new.h5"
raw_data = pd.read_hdf(path)

In [8]:
raw_columns = raw_data.columns[raw_data.columns.str.contains('x')].to_list()

In [11]:
slected_feature_indices = [raw_columns.index(i) for i in selected_feature]

In [12]:
np.save('slected_feature_indices.npy',slected_feature_indices)

In [13]:
selected_feature_indices = np.load('slected_feature_indices.npy')
train_x = train_x[selected_feature_indices,:,:]

In [9]:
#查看raw_data每支股票的缺失数据数量
rawdata = raw_data[raw_data['univ_tradable']==1]
date_list = raw_data.index.get_level_values('date').unique().to_list()
code_group_df = rawdata.groupby(level='code')
#查看code_group_df中每支股票的日期，有多少是比data_list少的
missing_values = code_group_df.apply(lambda x: len(date_list)-len(x.index.get_level_values('date').unique().to_list()))



In [10]:
#查看missing_values中缺失值小于data_list长度三分之一的股票
stock_universe=missing_values[missing_values<(len(date_list)/3)] #算上universe_tradable 为2836
code =stock_universe.index.to_list()
#把这些股票从raw_data中选出来
data = raw_data.loc[raw_data.index.get_level_values('code').isin(code)]

In [11]:
#查看data中包含的股票个数
stock_num = data.index.get_level_values('code').unique().to_list() #3355支股票
#如果这些股票在某个日期没有值，则对这个日期增加这支股票的股票代码为multi_index的第二个维度，并且每个特征填0
#首先构建一个新的multi_index
date_list = data.index.get_level_values('date').unique().to_list()
code_list = data.index.get_level_values('code').unique().to_list()
new_index = pd.MultiIndex.from_product([date_list,code_list],names=['date','code'])
#把data的index换成new_index
data = data.reindex(new_index)
#把nan填充为0
data = data.fillna(0) #1215*3355=4076325


In [16]:
#提出涨跌停的列
limit_up = data['ud_limit_h2'].to_frame()
limit_up.to_hdf('/home/laiminzhi/data/xydata/xy_data_2836_udlimit.h5',key='limit_up')

In [14]:
print(limit_up['ud_limit_h2'].value_counts())

ud_limit_h2
 0.0    3392017
 1.0      41297
-1.0      12426
Name: count, dtype: int64


In [6]:
# 确保DataFrame按照 (date, code) 排序
df = data.sort_index()

def replace_with_last_valid(group):
    # 标记需要替换的行
    mask = group['ud_limit_h2'] == 1
    group.loc[group['ud_limit_h2'] == 1, :] = np.nan
    # 使用前向填充（ffill）来填充标记的行
    group[mask] = group.ffill()
    
    # 返回处理后的组
    return group

# 应用函数
df = df.groupby(level='code').apply(replace_with_last_valid)

In [7]:
#存储data,此时的data是筛选出了3355支缺失值不太多的股票，并用0填充了缺失值，且把nan填充为0的数据
output_path = "/home/laiminzhi/data/xydata/xy_data_2836.h5"
df.to_hdf(output_path,key='data',mode='w')
"""
#另外把里面每一列用pivot_table转换成矩阵，然后存储
output_feature_path = "/home/laiminzhi/data/feature_data/"
for feature in data.columns:
    feature_df = data.pivot_table(index='date',columns='code',values=feature)
    #存为pickle文件
    feature_df.to_pickle(output_feature_path+feature+'.pkl')
"""

'\n#另外把里面每一列用pivot_table转换成矩阵，然后存储\noutput_feature_path = "/home/laiminzhi/data/feature_data/"\nfor feature in data.columns:\n    feature_df = data.pivot_table(index=\'date\',columns=\'code\',values=feature)\n    #存为pickle文件\n    feature_df.to_pickle(output_feature_path+feature+\'.pkl\')\n'

In [111]:
#读入df
df= pd.read_hdf('/home/laiminzhi/data/xydata/xy_data_2836.h5')
#把df转变为numpy数组，形状为(featrue_num,date_num,stock_num)
#去掉df中的无关列
universe = np.full(shape=(len(date_list),len(code_list)),fill_value=1)
timestep=30
res = np.full(shape=(universe.shape[0], universe.shape[1], timestep), fill_value=-1, dtype=int)

df.index = df.index.set_names(['code1','date','code'], level=[0, 1,2])

# 删除不需要的索引层，假设你想删除第一个 'code' 索引层
df = df.droplevel('code1')


In [112]:
#df =df.reset_index(level='date')
selected_cols = ['y1_label']+ df.columns[df.columns.str.startswith('x')].to_list()
df = df[selected_cols]
#用pivot_table把df转变为numpy数组
result_data = []
for col in df.columns:
    feature_table = df.pivot_table(index='date',columns='code',values=col)
    result_data.append(feature_table.values)

result_data = np.array(result_data) #(243,1215,2836)
print(result_data.shape)


(243, 1215, 2836)


In [200]:
np_data = result_data
print(df.loc['20180102']['x_1'])
print(np_data[0,:,:])

code
000001.SZ    0.118946
000002.SZ    0.158915
000006.SZ    0.000000
000009.SZ    0.386068
000010.SZ    0.447214
               ...   
603990.SH    0.000000
603993.SH    0.185461
603997.SH    0.000000
603998.SH    0.688238
603999.SH    0.000000
Name: x_1, Length: 2836, dtype: float64
[[-0.04173656 -0.01101659  0.         ... -0.00961667 -0.00119427
   0.01266947]
 [ 0.00231725  0.02722806  0.         ... -0.0054493   0.0335952
  -0.00385166]
 [-0.01129959  0.0212772   0.         ... -0.01263642  0.0044783
  -0.00031048]
 ...
 [ 0.00216399  0.00585184  0.02619968 ...  0.00410411  0.02044939
   0.03488302]
 [ 0.05149177  0.01068762 -0.04503276 ... -0.02305013  0.05776934
  -0.01444793]
 [ 0.03475181  0.01004841  0.00492698 ...  0.02694458  0.05776934
  -0.00984881]]


In [114]:
def get_slices(universe,timestep,res):
    for di in range(timestep-1, universe.shape[0]):
        ix = universe[di] == 1
        for ii in range(universe.shape[1]):
            if not ix[ii]:
                res[di, ii] = res[di - 1, ii]
            # 在前面universe不为0的日子中搜索该股票，找不到就保持-1
            # 使用np.where查找universe中的非零值
            idxs = np.where(universe[:di+1, ii] == 1)[0]
            if len(idxs) > 0:
                # 获取最近的`timestep`个非零值的日期
                if len(idxs) >= timestep:
                    recent_idxs = idxs[-timestep:]
                else:
                    recent_idxs = idxs[-len(idxs):]
                res[di, ii, -len(recent_idxs):] = recent_idxs
                # 更新res数组，如果recent_idxs长度小于timestep，则剩余部分保持-1                   
    return res #(1215,2836,30)
res = get_slices(universe,timestep,res)

In [202]:
def get_slices2(data,timestep,di):
#不需要universe，直接从已经对齐的(featrue,date,stock)数据中获取timestep的切片
#输出:(2836,243,30)
#要求输入的di是大于timestep-1的，且di在data.shape[1]范围内,di对应每个timestep的最后一天
    res = [None] * data.shape[2]
    if di < timestep-1:
        return np.zeros((data.shape[2],timestep))
    else:
        for ii in range(data.shape[2]):
            # 在前面universe不为0的日子中搜索该股票，找不到就保持-1
            # 使用np.where查找universe中的非零值
            
            values = data[:,(di+1)-timestep:(di+1),ii]#(243,timestep,1)
            res[ii] = values  
    res = np.array(res)
    res = res.transpose(0,2,1)             
    return np.array(res) #(2836,30,243)
test = get_slices2(np_data,timestep,29)

In [204]:
for di in range(29,data.shape[1]):
    test = get_slices2(np_data,timestep,di)

In [121]:
def get_feature_slices_for_all_stocks(feature, res, di):
    # 获取di对应的所有股票的日期索引
    date_indices = res[di, :, :]
    
    # 初始化结果数组，形状为(feature_nums, ii, timestep)，默认值为0
    #feature_slices = np.empty((feature.shape[0], feature.shape[2], timestep),dtype=object)
    feature_slices = np.zeros((feature.shape[0], feature.shape[2], timestep))
    
    # 对于每个时间步，进行向量化操作
    for t in range(timestep):
        # 获取当前时间步的日期索引
        current_date_indices = date_indices[:, t]
        
        # 过滤掉值为-1的索引
        valid_indices = current_date_indices != -1
        
        # 获取有效的日期索引和股票索引
        valid_date_indices = current_date_indices[valid_indices]
        valid_ii_indices = np.arange(len(valid_indices))[valid_indices]
        
        # 使用有效的日期索引和股票索引从feature中提取切片
        feature_slices[:, valid_ii_indices, t] = feature[:, valid_date_indices, valid_ii_indices]
        
    return feature_slices #(feature_nums, code_nums, timestep)

for di in range(0,len(date_list)):
    feature_slices = get_feature_slices_for_all_stocks(np_data, res, di)

In [120]:
print(feature_slices[0, :, :])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
first_diff_X = X.groupby(level='code').diff()


                    x_volline_all_1  x_volline_all_2  x_volline_all_3  \
date     code                                                           
20180102 000001.SZ         0.121064         0.983365         0.999183   
         000002.SZ         0.247414         1.011845         0.999566   
         000004.SZ         0.094779         0.859929         1.000172   
         000008.SZ         0.019896         0.848975         0.999186   
         000009.SZ         0.025538         0.981879         0.999630   

                    x_volline_all_4  x_culvol_pm_1  x_culvol_pm_2  \
date     code                                                       
20180102 000001.SZ         0.372051       0.204296       0.123011   
         000002.SZ         0.382665       0.212815       0.073555   
         000004.SZ         0.084023       0.000000       0.000000   
         000008.SZ         0.146017       0.596666       0.623502   
         000009.SZ         0.361936       0.425753       0.292394   

    

In [ ]:
#把一阶差分存下来
first_diff_X.to_hdf('/home/laiminzhi/data/xydata/first_diff_X_old.h5',key='data',mode='w')

In [2]:
#读取feature的pnl表现
feature_pnl = pd.read_csv("batch_results_rank_area.csv",index_col=0)

In [3]:
#以Area列排序，筛选出Area>20的feature
feature_pnl = feature_pnl.sort_values(by='Area',ascending=False)
feature_pnl_selected = feature_pnl[feature_pnl['Area']>20]

In [25]:
#把feature_pnl_selected的AlphaName列取出来，并去掉_output后缀，保留x_number的结构
feature_pnl_selected['AlphaName'] = feature_pnl_selected['AlphaName'].str.replace('_output','')
feature_name = feature_pnl_selected['AlphaName']
feature_name.to_csv('selected_feature.csv')

/tmp/ipykernel_35572/268157193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_pnl_selected['AlphaName'] = feature_pnl_selected['AlphaName'].str.replace('_output','')


In [26]:
selected_feature = pd.read_csv('selected_feature.csv',index_col=0)
selected_feature = selected_feature['AlphaName'].to_list()

In [10]:
def cal_alpha(XY):
    #查看pnl如何计算
    d1 = XY.copy()
    enterRatio = 0.9
    exitRatio = 0.9
    ## 1) calculate yestRank;
    d1['yestRank'] = d1.groupby('date')['yest'].rank(method='average',na_option='keep',ascending=True,pct=True)
    rtnMat = pd.pivot_table(data=d1,index='date',columns='code',values='y',dropna=False)
    yestMat = pd.pivot_table(data=d1,index='date',columns='code',values='yest',dropna=False)
    yestRankMat = pd.pivot_table(data=d1,index='date',columns='code',values='yestRank',dropna=False).fillna(0)
    posiMat = pd.DataFrame(np.full(yestRankMat.shape,fill_value=0),index=yestRankMat.index,columns=yestRankMat.columns)
    ud_limitMat = pd.pivot_table(data=d1,index='date',columns='code',values='ud_limit_h2',dropna=False).fillna(0)

    ## 2) calPosiMat： ## no buy if up_limit && no sell if down_limit;
    for i,row_index in enumerate(posiMat.index):
        if (i==0):
            continue
        flag1 = (yestRankMat.iloc[i,:]>enterRatio)
        flag2 = (posiMat.iloc[i-1,:]==0) & (ud_limitMat.iloc[i,:]==1)
        posiMat.loc[row_index,(~flag2 & flag1)] = 1

        flag3 = (yestRankMat.iloc[i,:]>exitRatio) & (yestRankMat.iloc[i,:]<=enterRatio)
        flag4 = (posiMat.iloc[i-1,:]==1)
        posiMat.loc[row_index,(flag3 & flag4)] = 1

        flag5 = (posiMat.iloc[i-1,:]==1) & (posiMat.iloc[i,:]==0) & (ud_limitMat.iloc[i,:]==-1)
        posiMat.loc[row_index,flag5] = 1
        
        if (i== (posiMat.shape[0]-1)):## position=0 if yest=NA on last day;
            flag6 = yestMat.iloc[i,:].isna()
            posiMat.loc[row_index,flag6] = 0


    pnlMat = rtnMat * posiMat
    pnlVec = pnlMat.sum(axis=1)/(posiMat==1).sum(axis=1)
    alpha = pnlVec.mean()*1e4
    return alpha

def get_XY(yest,xy):
    #xy是原始xy文件，yest是合并后的预测文件
    universe = 'univ_tradable'

    XY = xy.loc[xy[universe]==1,:'ud_limit_h4']
    XY= XY.rename(columns={'y1':'y'})
    XY = pd.merge(XY, yest,on=['date','code'],how='inner')

    ##---- 1. benchmark ----##
    XY['yest'] = XY['Y_hat']
    return XY

In [8]:
xy = pd.read_hdf('/home/laiminzhi/data/xydata/xy_data_3355.h5')

In [23]:
#另yest为xy中的x_62
universe = 'univ_tradable'
XY = xy.loc[xy[universe]==1,:'ud_limit_h4']
XY= XY.rename(columns={'y1':'y'})

##---- 1. benchmark ----##
XY['yest'] = xy['x_98']
XY.replace([np.inf, -np.inf], np.nan,inplace=True)
alpha = cal_alpha(XY)
print(alpha)

2.0710296776997246


In [2]:
#读取处理好的数据
test_index = np.load("/home/laiminzhi/new_data/test_data/lstm_30_y1_label__new_uniform_no_ud_limit_h2/test_index.npy",allow_pickle=True)


In [7]:
#查看共有多少支不一样的股票
unique_code = test_index[:,:,1].flatten()
#去除重复
unique_code = np.unique(unique_code)

In [10]:
def split_data(grouped_df,timestep):
        feature_index = []
        #遍历每个分组
        for stock_code, group_df in grouped_df:
            #对每个分组进行日期上的滚动分割
            for i in range(0, len(group_df) - timestep -1):  # 这里以timestep天为一个窗口
                #获取当前窗口的日期范围
                start_date = group_df.index[i]
                end_date = group_df.index[i + timestep-1]

                #使用 loc 获取当前窗口的数据，并将股票代码作为列名
                window_data = group_df.loc[start_date:end_date].reset_index(drop=False)
                window_index = window_data.loc[:,['date','code']].values

                feature_index.append(window_index)

        feature_index = np.array(feature_index)

        return feature_index


In [13]:
device = torch.device("cuda:2" if True and torch.cuda.is_available() else "cpu")